<a href="https://colab.research.google.com/github/rohitkhadka1/AI-ML-DS/blob/main/Backtesting_using_backtrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader

In [ ]:
from google.colab import files
files.upload()

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("nepsealpha_export_price_NABIL_2020-11-19_2025-11-19_adjusted.csv")

In [5]:
df.head()

,Symbol,Date,Open,High,Low,Close,Percent Change,Volume,Turn Over
0,NABIL,2025-11-19,511.0,511.0,505.0,507.0,-0.02 %,"18,121.00",-
1,NABIL,2025-11-18,508.0,510.0,505.6,507.1,-0.18 %,"24,383.00",-
2,NABIL,2025-11-17,504.0,509.9,504.0,508.0,0.89 %,"24,130.00",-
3,NABIL,2025-11-16,503.0,506.9,501.0,503.5,0.10 %,"31,462.00",-
4,NABIL,2025-11-13,505.1,505.1,502.0,503.0,-0.20 %,"32,775.00",-


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          1169 non-null   object 
 1   Date            1169 non-null   object 
 2   Open            1169 non-null   float64
 3   High            1169 non-null   float64
 4   Low             1169 non-null   float64
 5   Close           1169 non-null   float64
 6   Percent Change  1169 non-null   object 
 7   Volume          1169 non-null   object 
 8   Turn Over       1169 non-null   object 
dtypes: float64(4), object(5)
memory usage: 82.3+ KB


In [7]:
df.drop(columns = ['Symbol', 'Percent Change', 'Turn Over'], inplace = True)

In [8]:
df['Volume'] = (
    df['Volume']
    .astype(str)
    .str.replace(',', '')
    .astype(float)
)
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1169 non-null   datetime64[ns]
 1   Open    1169 non-null   float64       
 2   High    1169 non-null   float64       
 3   Low     1169 non-null   float64       
 4   Close   1169 non-null   float64       
 5   Volume  1169 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 54.9 KB


In [9]:
df.set_index('Date', inplace = True)
df.sort_index(inplace = True)
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-19,428.669045,450.906843,425.830177,428.195900,94311.0
2020-11-22,435.293070,466.047472,435.293070,456.584579,223043.0
2020-11-23,458.950302,470.305774,458.477157,462.735459,75761.0
2020-11-24,473.144641,473.144641,454.218856,459.896591,58491.0
2020-11-25,464.154893,486.392691,459.896591,484.973257,232466.0


In [10]:
df1 = df.copy()

In [14]:
import backtrader as bt
cerebro = bt.Cerebro()
data = bt.feeds.PandasData(
    dataname=df1,
    open='Open',
    high='High',
    low='Low',
    close='Close',
    volume='Volume',
    openinterest=None
)


In [17]:
type(data)

backtrader.feeds.pandafeed.PandasData

In [18]:
cerebro.adddata(data)


In [19]:
#Simple Crossover Strategy
class SMACrossover(bt.Strategy):
    params = dict(
        fast=20,
        slow=50
    )

    def __init__(self):
        sma_fast = bt.indicators.SMA(self.data.close, period=self.p.fast)
        sma_slow = bt.indicators.SMA(self.data.close, period=self.p.slow)

        self.crossover = bt.indicators.CrossOver(sma_fast, sma_slow)

    def next(self):
        if not self.position and self.crossover > 0:
            self.buy()
        elif self.position and self.crossover < 0:
            self.sell()


In [20]:
#Buy&Hold Benchmark
class BuyAndHold(bt.Strategy):
    def next(self):
        if not self.position:
            self.buy()


In [26]:
#run backtest
def run_backtest(strategy, df, strategy_name):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)

    cerebro.broker.setcash(100000)
    cerebro.broker.setcommission(commission=0.0015)

    data = bt.feeds.PandasData(
        dataname=df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=None
    )
    print("ran")

    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    results = cerebro.run()
    strat = results[0]

    print(f"\n--- {strategy_name} ---")
    print("Final Portfolio Value:", cerebro.broker.getvalue())
    print("Sharpe Ratio:", strat.analyzers.sharpe.get_analysis())
    print("Max Drawdown:", strat.analyzers.drawdown.get_analysis()['max']['drawdown'])
    print("Total Return:", strat.analyzers.returns.get_analysis()['rtot'])

    cerebro.plot()



In [27]:
run_backtest(SMACrossover, df, "SMA Crossover (20, 50)")
run_backtest(BuyAndHold, df, "Buy & Hold")


ran

--- SMA Crossover (20, 50) ---
Final Portfolio Value: 99812.90514327933
Sharpe Ratio: OrderedDict({'sharperatio': -0.6538143307647934})
Max Drawdown: 0.42118758464276473
Total Return: -0.0018727009775981027


<IPython.core.display.Javascript object>

ran

--- Buy & Hold ---
Final Portfolio Value: 100071.05399034577
Sharpe Ratio: OrderedDict({'sharperatio': -0.3598744877336273})
Max Drawdown: 0.6183536720650203
Total Return: 0.0007102875894928573
